Загрузка файла ./dataset/positive.csv

In [2]:
from google.colab import files
uploaded_pos = files.upload()

Saving positive100.csv to positive100.csv


Загрузка файла ./dataset/negative.csv

In [4]:
uploaded_neg = files.upload()

Saving negative100.csv to negative100.csv


### Импорт необходимых библиотек

In [5]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import seaborn as sns
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Считывание данных

In [6]:
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

### Формирование сбалансированного датасета

In [7]:
sample_size = 40000
reviews_withoutshuffle = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels_withoutshuffle = np.asarray([1] * sample_size + [0] * sample_size)

assert len(reviews_withoutshuffle) == len(labels_withoutshuffle)
from sklearn.utils import shuffle
reviews,labels = shuffle(reviews_withoutshuffle, labels_withoutshuffle, random_state=0)

In [9]:
print(reviews[100])

@Nazirochka03 у меня всееегда так :D я не смотрю в глаза называя цифры


### Токенизация

In [12]:
def tokenize():
  punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'
  all_reviews = 'separator'.join(reviews)
  all_reviews = all_reviews.lower()
  all_text = ''.join([c for c in all_reviews if c not in punctuation])
  texts_split = all_text.split('separator')
  all_text = ' '.join(texts_split)
  words = all_text.split()
  return words, texts_split

words, texts_split = tokenize()

In [13]:
new_reviews = []
for review in texts_split:
    review = review.split()
    new_text = []
    for word in review:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    new_reviews.append(new_text)

In [15]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii - 1 for ii, word in enumerate(vocab, 1)}
reviews_ints = []
for review in new_reviews:
    reviews_ints.append([vocab_to_int[word] for word in review])


In [16]:
def add_pads(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    for i, row in enumerate(reviews_ints):
        if len(row) == 0:
            continue
        features[i, -len(row):] = np.array(row)[:seq_length]
    return features

### Разделение на обучающую, валидационную и тестовую выборки

In [17]:
features = add_pads(reviews_ints, seq_length=30)
split_frac = 0.8 # 80% на обучающую выборку

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = labels[:split_idx], labels[split_idx:]
test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
batch_size = 50
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

### Определение режима: GPU или CPU

In [18]:
train_gpu=torch.cuda.is_available()

print(train_gpu)

True


### Архитектура нейронной сети

In [19]:
import torch.nn as nn

class LSTM_architecture(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, number_of_layers, drop=0.5):
        super(LSTM_architecture, self).__init__()
        self.output_size = output_size
        self.number_of_layers = number_of_layers
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, number_of_layers,dropout=drop, batch_first=True)
        self.dropout = nn.Dropout(0.45)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, hidden_state):
        lstm_out, hidden_state = self.lstm(self.embedding(x.long()), hidden_state)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sigmoid(out)
        batch_size = x.size(0)
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]
        return sig_out, hidden_state

    def init_hidden_state(self, batch_size):
        weight = next(self.parameters()).data
        hidden_state = (weight.new(self.number_of_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.number_of_layers, batch_size, self.hidden_dim).zero_())
        return hidden_state


###Выбор гиперпараметров и инициализация сети

In [20]:
vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 100
hidden_dim = 128
n_layers = 2
model = LSTM_architecture(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Обучение модели

In [21]:
epochs = 4 #оптимальное количество эпох для того, чтобы модель достаточно обучилась, но не переобучилась
counter = 0
batch_num = 100
clip=5 
if(train_gpu):
    model.cuda()
num_correct = 0
model.train()
for e in range(epochs):
    h = model.init_hidden_state(batch_size)
    for inputs, labels in train_loader:
        num_correct = 0
        counter += 1
        if(train_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
        h = tuple([each.data for each in h])
        model.zero_grad()
        output, h = model.forward(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        if counter % batch_num == 0:
            val_h = model.init_hidden_state(batch_size)
            val_losses = []
            model.eval()
            for inputs, labels in valid_loader:
                val_h = tuple([each.data for each in val_h])
                if(train_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())
                val_losses.append(val_loss.item())

                #accuracy
                pred = torch.round(output.squeeze()) 
                correct_tensor = pred.eq(labels.float().view_as(pred))
                correct = np.squeeze(correct_tensor.numpy()) if not train_gpu else np.squeeze(correct_tensor.cpu().numpy())
                num_correct += np.sum(correct)
                valid_acc = num_correct/len(valid_loader.dataset)

            model.train()
            print("Epoch: {} ;".format(e+1),
                  "Batch Number: {};".format(counter),
                  "Train Loss: {:.4f} ;".format(loss.item()),
                  "Valid Loss: {:.4f} ;".format(np.mean(val_losses)),
                  "Valid Accuracy: {:.4f}".format(valid_acc))


Epoch: 1 ; Batch Number: 100; Train Loss: 0.7056 ; Valid Loss: 0.6774 ; Valid Accuracy: 0.5734
Epoch: 1 ; Batch Number: 200; Train Loss: 0.6054 ; Valid Loss: 0.6379 ; Valid Accuracy: 0.6111
Epoch: 1 ; Batch Number: 300; Train Loss: 0.5916 ; Valid Loss: 0.6249 ; Valid Accuracy: 0.6388
Epoch: 1 ; Batch Number: 400; Train Loss: 0.6366 ; Valid Loss: 0.6163 ; Valid Accuracy: 0.6460
Epoch: 1 ; Batch Number: 500; Train Loss: 0.5793 ; Valid Loss: 0.6130 ; Valid Accuracy: 0.6526
Epoch: 1 ; Batch Number: 600; Train Loss: 0.6658 ; Valid Loss: 0.5995 ; Valid Accuracy: 0.6594
Epoch: 1 ; Batch Number: 700; Train Loss: 0.6515 ; Valid Loss: 0.6027 ; Valid Accuracy: 0.6661
Epoch: 1 ; Batch Number: 800; Train Loss: 0.5797 ; Valid Loss: 0.5912 ; Valid Accuracy: 0.6784
Epoch: 1 ; Batch Number: 900; Train Loss: 0.5161 ; Valid Loss: 0.5851 ; Valid Accuracy: 0.6734
Epoch: 1 ; Batch Number: 1000; Train Loss: 0.6860 ; Valid Loss: 0.5785 ; Valid Accuracy: 0.6893
Epoch: 1 ; Batch Number: 1100; Train Loss: 0.5490

### Предсказание окраски по тексту

In [26]:
def tokenize_text(test_review):
    test_review = test_review.lower()
    punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'
    test_text = ''.join([c for c in test_review if c not in punctuation])
    test_words = test_text.split()
    
    new_text = []
    for word in test_words:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in new_text])

    return test_ints

In [30]:
def predict(model, test_review, sequence_length=30):
    model.eval()
    test_ints = tokenize_text(test_review)
    seq_length=sequence_length
    features = add_pads(test_ints, seq_length)
    feature_tensor = torch.from_numpy(features)
    batch_size = feature_tensor.size(0)
    h = model.init_hidden_state(batch_size)
    if(train_gpu):
        feature_tensor = feature_tensor.cuda()
    output, h = model(feature_tensor, h)
    
    pred = torch.round(output.squeeze()) 
    print('Вероятность положительного ответа {:.6f}'.format(output.item()))
    
    if(pred.item()==1):
        print("Позитивное сообщение")
    else:
        print("Негативное сообщение")

In [32]:
seq_length = 30 
test_review = "Как ты могла со мной так мерзко поступить? Я ухожу!"
predict(model, test_review, seq_length)


Вероятность положительного ответа 0.007588
Негативное сообщение
